# Introduction
This tutorial allows you to compute the velocity and pressure field caused by a density anomaly of ellipsoidal shape within a fluid by solving the incompressible Stokes equation for laminar flow. 
This example is called the Stokes sphere problem. It uses a finite element approach to solve the Stokes equation (force balance, or conservation of momentum) based on a package call FEniCSx.

The tutotrial will first describe two ways to obtain the computing environment for FEniCSx and the then solve the problem. We can obtain FEniCSx by installing it via package manager Anaconda or using its docker image. 
More methods could be found via the official FEniCS/dofinx GitHub repository's README.md https://github.com/FEniCS/dolfinx/blob/main/README.md.
The Stokes solve apprroach itself is modified from the official FEniCSx Sotkes demo via https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_stokes.html

# FEniCSx installation with Anaconda
For Windows users, please install Anaconda through Windows Linux Subsystem. You can install Ubuntu from Microsoft Store. 

For MacOS and Linux users, Anaconda installation package can be found here https://www.anaconda.com/products/distribution#Downloads.

For example, for Linux (or Windows Linux Subsystem) users, 
```
wget https://repo.anaconda.com/archive/Anaconda3-2022.10-Linux-x86_64.sh
bash Anaconda3-2022.10-Linux-x86_64.sh
```

For MacOS users, 
```
wget https://repo.anaconda.com/archive/Anaconda3-2022.10-MacOSX-x86_64.sh
bash Anaconda3-2022.10-MacOSX-x86_64.sh
```

With Anaconda installed, you can install FEniCSx using fenicsx-env.yml and using the following command:

```
conda env create -f fenicsx-env.yml
```
It will create an Anaconda environment named fenicsx.

For Ubuntu (Windows Linux Subsystem) users, you may need to run this command:
```
sudo apt install libgl1-mesa-glx xvfb
```
This will allow pyvista to plot.

# Activate/deactivate a conda environment
By following the above steps, FEniCSx and Jupyter Notebook is installed inside the conda environment fenicsx-env.

To activate it, 
```
conda activate fenicsx-env
```

To deactivate it
```
conda deactivate
```

# Start the Jupyter Notebook from terminal
Becuase Jupyter Notebook is installed inside the conda environment fenicsx-env, you need to activate fenicsx-env first. Then, type the following command 
```
jupyter notebook --allow-root
```

You will get a website link. Copy and paste that link to a browser should start you the Jupyter Notebook.

# FEniCSx installation with Docker
Docker Desktop can be downloaded and installed from this link https://www.docker.com/products/docker-desktop/. 

With Docker installed, please type the following command in the terminal:
```
docker run --init -ti -p 8888:8888 dolfinx/lab:stable  # Access at http://localhost:8888
```

Then, Jupyter Notebook should automatically start at port 8888. 

You can stop the container and restart it later. Then, you will see Jupyter Notebook requires a password/Token for your Jupyter server. 

Open a terminal inside the docker container, type
```
Jupyter server list
```

Copy and paste the Token (a long list of charaters following https://) to the webbrowser and set up a password. Later, you just need to type in the password to log in the Jupyter Notebook.

In [1]:
# loading packages
import numpy as np
import ufl
from dolfinx.fem import (Constant, Function, FunctionSpace, dirichletbc,
                         extract_function_spaces, form,
                         locate_dofs_topological, locate_dofs_geometrical)
from dolfinx.io import XDMFFile
from dolfinx.mesh import CellType, create_rectangle, locate_entities_boundary
from mpi4py import MPI
from petsc4py import PETSc
from ufl import div, dx, grad, inner
from dolfinx import fem, la
from dolfinx.io import gmshio
from math import *
from dataclasses import dataclass,astuple,asdict # for C-like structure

# model description: a top rectangle + a bottom rectangle + an elliptical inclusion inside the bottom rectangle.
# ---
# boundary descriptions:
# left boundary  : A
# bottom boundary: B
# right boundary : C
# top boundary   : D
# ---
# material properties:
@dataclass
class parameters: 
    # Adjustable parameters
    eta_crust  = 1.  # viscosity of the upper block.
    eta_mantle = 1.  # viscosity of the bottom block.
    eta_inc    = 10  # viscosity of the elliptical anomaly. 

    dgrav      = 10. # delta gravity anomaly for the ellipse. 

    # ----------------------------------
    # adjustable parameters for the mesh.
    dxx      = 0.1 # cell size near boundary points.
    dxx_curv = 10  # cell numbers to capture the ellipitical curvatures. More will increase the resolution near the vertexes of the ellipse.

    w   = 2.  # width of the model
    h1  = 1.  # height of the bottom rectangle
    h2  = 0.5 # heigth of the top rectangle

    xe  = w/2;  # x cooridnate of the ellipse center
    ye  = h1/2; # y coordinate of the ellipse center
    # the center of the ellipse is at the center of the bottom rectangle at (xe,ye)=(w/2,h1/2).
    ra  = 0.4 # a of the ellipse
    ratio_a_b = 4 # ratio of a/b of the ellipse
    rb  = ra/ratio_a_b # b of the ellipse

    rot = 0.5*pi # rotation of the ellipse counterclockwise around its center in radians.

    # ----------------------------------
    bcs_type  = 1   # please choose the boundary conditions and the options could be expanded. 
    # 1: free slip for all surfaces A-D;
    # 2: free slip for A-C, D free.
    # 3: no slip for B, fixed velocity for D, no y motion + tao_xy=0 for A and C. 
    # --- for visualization
    if bcs_type == 1:
        velocity_vector_scale_factor = 2.
    elif bcs_type == 2:
        velocity_vector_scale_factor = 0.4
    elif bcs_type == 3:
        velocity_vector_scale_factor = 0.2

par = parameters()
velocity_vector_scale_factor = par.velocity_vector_scale_factor

In [2]:
from StokesLib import gmsh_stokes_tutorial, fem_main, block_iterative_solver
import pyvista
from dolfinx import plot

# sovle the system
crust_marker, mantle_marker, inc_marker, \
left_marker, right_marker, top_marker, bottom_marker = gmsh_stokes_tutorial(par)
a, a_p, L, bcs, V, Q, msh = fem_main(crust_marker, mantle_marker, inc_marker, 
                                     left_marker, right_marker, top_marker, bottom_marker,
                                     par)
norm_u_1, norm_p_1, u_, p_ = block_iterative_solver(a, a_p, L, bcs, V, Q, msh)

# visualize results using pyvista
pyvista.start_xvfb()
# create a VTK compatible mesh with function space V for velocity.
topology, cell_types, geometry = plot.create_vtk_mesh(V) 
values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
values[:, :len(u_)] = u_.x.array.real.reshape((geometry.shape[0], len(u_)))

# Create a point cloud of glyphs
grid1 = pyvista.UnstructuredGrid(topology, cell_types, geometry)
grid1["u"] = values
glyphs = grid1.glyph(orient="u", factor=velocity_vector_scale_factor)

# Create plotter
plotter = pyvista.Plotter()
#plotter.add_text("Velocity", font_size=14, color="black", position="upper_edge")
plotter.add_mesh(grid1, style="wireframe", color="k")
plotter.add_mesh(glyphs)
plotter.view_xy()

grid1.save('a.vtk')
if pyvista.OFF_SCREEN:
    fig_as_array = plotter.screenshot("velocity.png", transparent_background=transparent,
                          window_size=[figsize, figsize])
else:
    window_size = [800, 900]
    plotter.show(window_size=window_size)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 40%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 5 (Line)
Info    : [ 70%] Meshing curve 6 (Line)
Info    : [ 80%] Meshing curve 7 (Line)
Info    : [ 90%] Meshing curve 8 (Ellipse)
Info    : Done meshing 1D (Wall 0.329083s, CPU 0.326519s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0797386s, CPU 0.063872s)
Info    : 428 nodes 905 elements
Info    : Writing 'stokes.msh'...
Info    : Done writing 'stokes.msh'
Info    : Reading 'stokes.msh'...
Info    : 18 entities
Info    : 428 nodes
Info    : 854 elements
Info    : Done reading 'stokes.msh'
(B) Norm of velocity coefficient vector (blocked, iterative): 1.558546946216387
(B) Norm of

In [4]:
# to visualize the function p_, we create a VTK-compatible grid.
# https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_pyvista.html
topology2, cell_types2, geometry2 = plot.create_vtk_mesh(Q) # Q is the function space for pressure.
grid2 = pyvista.UnstructuredGrid(topology2, cell_types2, geometry2)
# for scalar plots, use .point_data
grid2.point_data["p"] = p_.x.array

grid2.set_active_scalars("p")
warped = grid2.warp_by_scalar() # using the warp function.

subplotter = pyvista.Plotter(shape=(1, 1)) # shape defines how many panels you want for subplots
subplotter.subplot(0, 0)
subplotter.add_text("Pressure", font_size=14, color="black", position="upper_edge")
subplotter.add_mesh(grid2, show_edges=False, show_scalar_bar=True)
subplotter.view_xy()

#subplotter.subplot(0, 1)
#subplotter.add_text("Warped function", position="upper_edge", font_size=14, color="black")
#sargs = dict(height=0.8, width=0.1, vertical=True, position_x=0.05,
#             position_y=0.05, fmt="%1.2e", title_font_size=40, color="black", label_font_size=25)
#subplotter.set_position([-3, 2.6, 0.3])
#subplotter.set_focus([3, -1, -0.15])
#subplotter.set_viewup([0, 0, 1])
#subplotter.add_mesh(warped, show_edges=True, scalar_bar_args=sargs)
if pyvista.OFF_SCREEN:
    subplotter.screenshot("pressure.png", transparent_background=transparent,
                          window_size=[figsize, figsize])
else:
    window_size = [800, 900]
    subplotter.show(window_size=window_size)